# Checking Previous Crawl Data

Checking if I still have the XML versions of the threads of Known-Item request posts.

In [2]:
import glob
import os

# XML threads are here
thread_dir = '/Volumes/Samsung_T5/Data/Projects/SBS/LibraryThing/threads/english-xml-threads/'
xml_files = glob.glob(os.path.join(thread_dir, '**/*.xml'))

# how many?
len(xml_files)

108915

In [16]:
xml_files[:10]
xf = xml_files[0]
file_map = {os.path.split(xf)[1].replace('.xml', ''): xf for xf in xml_files}

In [30]:
import pandas as pd

# use the label file to extract the LibraryThing thread IDs
label_file = '/Users/marijnkoolen/writing/Humanities-Cluster/Papers/2022/JDoc-2022/Labels-all_domains-no_double.csv'
df = pd.read_csv(label_file, sep='\t')

all_post_ids = df[(df.Domain == 'Books')].Post_id.unique()
ki_post_ids = df[(df.Domain == 'Books') & (df['Known-item'] == 1)].Post_id.unique()
len(post_ids)

file_map = {os.path.split(xf)[1].replace('.xml', ''): xf for xf in xml_files}
post_files = {post_id: file_map[post_id] for post_id in post_id if post_id in file_map}
for post_id in post_ids:
    if post_id not in file_map:
        print(post_id)

First, copy threads of all annotated posts to the data dir:

In [31]:
xml_dir = '../../data/books/All_posts_thread_xml/'

if os.path.exists(xml_dir) is False:
    os.mkdir(xml_dir)

for post_id in post_ids:
    if post_id not in file_map:
        continue
    
    !cp {file_map[post_id]} $xml_dir

Second, copy threads of all KI posts to a separate sub-dir:

In [ ]:
thread_dir = '../../data/books/All_posts_thread_xml/'
xml_files = glob.glob(os.path.join(thread_dir, '*.xml'))
len(xml_files)
xml_dir = '../../data/books/KI_thread_xml'

if os.path.exists(xml_dir) is False:
    os.mkdir(xml_dir)

for post_id in post_ids:
    if post_id not in file_map:
        continue
    
    !cp {file_map[post_id]} $xml_dir

# Crawling Solved Known Item Topics

In [1]:
import requests

base_url = 'https://www.librarything.com/ngroups/724/Name-that-Book'

# response = requests.get(base_url)
# response.status_code

## Reading Downloaded Pages

In [2]:
import glob
import os


download_dir = '../data/LT_Name_that_Book_topic_lists//'

topic_list_files = glob.glob(os.path.join(download_dir, '*.html'))
topic_list_files = sorted(topic_list_files, key=lambda tf: int(tf.split('-')[-1][:-5]))
len(topic_list_files)

160

In [3]:
from parse import parse_time_string

rel_date = "Monday 8:40pm"
topic_file = '../data/LT_Name_that_Book_topic_lists/Name that Book _ LibraryThing-1.html'

print(parse_time_string(rel_date, topic_file))

rel_date = "Yesterday 8:40pm"
print(parse_time_string(rel_date, topic_file))

rel_date = "January 10"
print(parse_time_string(rel_date, topic_file))

2024-07-08 15:07:00
2024-07-10 15:07:00
2024-01-10 00:00:00


In [4]:
from parse import parse_topic_page

In [5]:
import pandas as pd


threads = []
for tl_file in topic_list_files:
    #print(tl_file)
    tl_threads = parse_topic_page(tl_file)
    threads.extend(tl_threads)

df = pd.DataFrame(threads)
df.shape, df[df.found == True].shape

((15996, 7), (2098, 7))

In [6]:
df

,topic_number,topic_title,user,last_post_timestamp,crawl_timestamp,num_posts,found
0,361924,Modern Fantasy where a Vegas HS guy takes a ch...,MissSquish,2024-07-11 07:51:40,2024-07-11T14:32:02.063697,2,False
1,361923,American Revolution Romance??,Christi295,2024-07-10 07:51:40,2024-07-11T14:32:02.063818,1,False
2,361922,Series of Young adult fiction set in Pacific N...,tantivy,2024-07-10 07:51:40,2024-07-11T14:32:02.063920,1,False
3,361910,Found: JA/YA Thriller Series with FMC,mobrxwn,2024-07-10 07:51:40,2024-07-11T14:32:02.064011,3,True
4,361921,Novel where a child meets 3 different versions...,Pacmane,2024-07-10 07:51:40,2024-07-11T14:32:02.064102,1,False
...,...,...,...,...,...,...,...
15991,304370,i need to find this book,cyzag,2019-03-11 00:00:00,2024-07-11T14:32:06.085223,2,False
15992,304384,Can't find this Drama Romance novel,anny.ana,2019-03-11 00:00:00,2024-07-11T14:32:06.085257,1,False
15993,304226,Title for the Romance Novel Plz,Marie79,2019-03-11 00:00:00,2024-07-11T14:32:06.085290,5,False
15994,304371,YA Fantasy: Boy that speaks symbols,BlackDragon0813,2019-02-11 00:00:00,2024-07-11T14:32:06.085326,1,False


In [7]:
df['topic_url'] = df.topic_number.apply(lambda x: f"https://librarything.com/topic/{x}")

In [8]:
df[df.found == True].shape

(2098, 8)

In [10]:
#df.to_csv('../data/thread_info-Name_that_Book.tsv', sep='\t', index=False)
df = pd.read_csv('../data/thread_info-Name_that_Book.tsv', sep='\t')
df

,topic_number,topic_title,user,last_post_timestamp,crawl_timestamp,num_posts,found,topic_url
0,361924,Modern Fantasy where a Vegas HS guy takes a ch...,MissSquish,2024-07-11 07:51:40,2024-07-11T14:32:02.063697,2,False,https://librarything.com/topic/361924
1,361923,American Revolution Romance??,Christi295,2024-07-10 07:51:40,2024-07-11T14:32:02.063818,1,False,https://librarything.com/topic/361923
2,361922,Series of Young adult fiction set in Pacific N...,tantivy,2024-07-10 07:51:40,2024-07-11T14:32:02.063920,1,False,https://librarything.com/topic/361922
3,361910,Found: JA/YA Thriller Series with FMC,mobrxwn,2024-07-10 07:51:40,2024-07-11T14:32:02.064011,3,True,https://librarything.com/topic/361910
4,361921,Novel where a child meets 3 different versions...,Pacmane,2024-07-10 07:51:40,2024-07-11T14:32:02.064102,1,False,https://librarything.com/topic/361921
...,...,...,...,...,...,...,...,...
15991,304370,i need to find this book,cyzag,2019-03-11,2024-07-11T14:32:06.085223,2,False,https://librarything.com/topic/304370
15992,304384,Can't find this Drama Romance novel,anny.ana,2019-03-11,2024-07-11T14:32:06.085257,1,False,https://librarything.com/topic/304384
15993,304226,Title for the Romance Novel Plz,Marie79,2019-03-11,2024-07-11T14:32:06.085290,5,False,https://librarything.com/topic/304226
15994,304371,YA Fantasy: Boy that speaks symbols,BlackDragon0813,2019-02-11,2024-07-11T14:32:06.085326,1,False,https://librarything.com/topic/304371


In [219]:
df[df.found == True]

,topic_number,topic_title,user,last_post_timestamp,crawl_timestamp,num_posts,found,topic_url
2,344039,Found: YA Action Story,smavs,September 2022,2024-07-11T09:53:48.062513,3,True,https://librarything.com/topic/344039
3,344021,Found: Magic Academy with Portals,RosetheReader,September 2022,2024-07-11T09:53:48.062539,15,True,https://librarything.com/topic/344021
10,344003,Found: action/mystery,konallis,September 2022,2024-07-11T09:53:48.062715,6,True,https://librarything.com/topic/344003
24,343974,Found: YA fantasy/romance,Kat-Stewart,September 2022,2024-07-11T09:53:48.063059,5,True,https://librarything.com/topic/343974
32,343853,Found: 80s Kids book about witches,LukeAnd,August 2022,2024-07-11T09:53:48.063256,3,True,https://librarything.com/topic/343853
...,...,...,...,...,...,...,...,...
15875,132804,Found: Children’s book Princess and dragon gir...,Caramellunacy,March 2021,2024-07-11T09:53:53.655923,4,True,https://librarything.com/topic/132804
15879,330386,Found: Series about two boys traveling the wor...,pattisimit,March 2021,2024-07-11T09:53:53.655983,2,True,https://librarything.com/topic/330386
15882,330133,Found: Horror: Alien who collects brains,hello10101,March 2021,2024-07-11T09:53:53.656029,3,True,https://librarything.com/topic/330133
15893,330330,"Found: popular author, thriller, kidnapped chi...",Caramellunacy,March 2021,2024-07-11T09:53:53.656197,4,True,https://librarything.com/topic/330330


In [165]:
df[df.found == True].last_post_timestamp.unique()

array(['September 2022', 'August 2022', 'January 21', 'January 20',
       'January 19', 'January 17', 'January 16', 'January 15',
       'January 13', 'April 2022', 'March 2022', 'June 2023',
       'January 2021', 'March 2021', 'February 2021', 'September 2021',
       'March 2020', 'June 2022', 'August 2023', 'August 2021',
       'November 2021', 'October 2021', 'April 2023', 'March 2023',
       'July 2023', 'Yesterday 8:53pm', 'Yesterday 6:04pm',
       'Yesterday 3:22pm', 'Yesterday 11:08am', 'Sunday 7:00pm',
       'Sunday 5:41pm', 'Sunday 4:51pm', 'Saturday 2:56pm',
       'Friday 4:58pm', 'Friday 12:12pm', 'July 2', 'July 1', 'June 29',
       'June 25', 'June 24', 'June 23', 'August 2020', 'July 2022',
       'May 2023', 'January 11', 'January 10', 'January 9', 'January 8',
       'January 7', 'January 4', 'January 3', 'January 2', 'October 2022',
       'November 2023', 'October 2023', 'April 19', 'April 17',
       'April 16', 'April 15', 'April 14', 'April 12', 'April 11'

In [225]:
df[df.found == True].topic_number.sort_values()#.nunique()

15268      5514
3139      50286
2681      50289
11919     54420
2064      57462
          ...  
2442     361846
2435     361862
2410     361906
2403     361910
2405     361916
Name: topic_number, Length: 2098, dtype: int64

In [13]:
found_urls = list(df[df.found].topic_url)

thread_dir = '../data/LT_Name_that_Book_threads/'

thread_files = [tf for tf in glob.glob(os.path.join(thread_dir, '*')) if 'crawl.sh' not in tf]
len(thread_files)

2096

In [14]:
downloaded_threads = [int(os.path.split(tf)[-1]) for tf in thread_files]
print(f"number of downloaded threads: {len(downloaded_threads)}")


number of downloaded threads: 2096


In [15]:

to_download_urls = list(df[(df.topic_number.isin(downloaded_threads) == False) & (df.found == True)].topic_url)

print(f"number of to download threads: {len(to_download_urls)}")

with open('../data/to_download_urls.txt', 'wt') as fh:
    for url in to_download_urls:
        fh.write(f"{url}\n")
#df.topic_number

number of to download threads: 0


In [58]:
import time

found_url = found_urls[0]

response = requests.get(found_url)


'https://librarything.com/topic/361803'

## Parse Thread Files

In [11]:
thread_dir = '../data/LT_Name_that_Book_threads/'

thread_files = [tf for tf in glob.glob(os.path.join(thread_dir, '*')) if 'crawl.sh' not in tf]
thread_files = sorted(thread_files, key=lambda tf: int(os.path.split(tf)[-1].split('-')[-1]), reverse=True)
thread_files[0]

'../data/LT_Name_that_Book_threads/361916'

In [14]:
from parse import parse_time_string

rel_date = "Monday 8:40pm"
topic_file = '../data/LT_Name_that_Book_topic_lists/Name that Book _ LibraryThing-1.html'

print(parse_time_string(rel_date, topic_file))

rel_date = "Yesterday 8:40pm"
print(parse_time_string(rel_date, topic_file))

rel_date = "January 10"
print(parse_time_string(rel_date, topic_file))

2024-07-08 15:07:00
2024-07-10 15:07:00
2024-01-10 00:00:00


In [39]:
import dateutil.parser as parser

from bs4 import BeautifulSoup


def extract_post_num_username(post_div):
    for a in post_div.find_all('a'):
        if 'class' in a.attrs and 'b' in a.attrs['class']:
            post_num = int(a.text)
        if 'href' in a.attrs and a.attrs['href'].startswith('/profile/'):
            username = a.text
    return post_num, username


def parse_post(post_div):
    post_info = {}
    post_info['message_num'] = int(post_div.attrs['id'].replace('fp', ''))
    if 'id' not in post_div.attrs:
        raise KeyError(f"no 'id' attribute in fp div: {div.attrs}")
    post_info['post_num'], post_info['username'] = extract_post_num_username(post_div)
    for sub_div in post_div.find_all('div'):
        #print('\nSUB_DIV:', sub_div)
        if 'mi' in sub_div.attrs['class']:
            time_string = sub_div.text.replace('Edited: ', '')
            post_info['edited'] = 1 if sub_div.text.startswith('Edited: ') else 0
            post_info['timestamp'] = parse_time_string(time_string, thread_file).isoformat()
        if 'mT' in sub_div.attrs['class']:
            post_info['text'] = ''
            post_info['touchstones'] = [] 
            post_info['external_resources'] = [] 
            for child in sub_div:
                if child.name == 'br':
                    post_info['text'] += '\n'
                elif child.name == 'a' and child.attrs['href'].startswith('/author'):
                    touchstone = {
                        'type': 'author',
                        'author_url': child.attrs['href'],
                        'author_name': child.text
                    }
                    post_info['touchstones'].append(touchstone)
                elif child.name == 'a' and child.attrs['href'].startswith('/work'):
                    touchstone = {
                        'type': 'work',
                        'work_url': child.attrs['href'],
                        'work_id': child.attrs['data-workid'],
                        'work_name': child.text
                    }
                    post_info['touchstones'].append(touchstone)
                elif child.name == 'a' and child.attrs['href'].startswith('http'):
                    external_resource = {
                        'type': 'external_resource',
                        'resource_url': child.attrs['href'],
                        'resource_name': child.text
                    }
                    post_info['external_resources'].append(external_resource)
                elif child.name == None:
                    post_info['text'] += child.text
                else:
                    print(f'CHILD: {child.name}\t#{child.text}#')
                    if child.name is not None:
                        print(f"\t{child.attrs}")
        #print('\nPOST_INFO: ', post_info)
        
    return post_info


thread_file = thread_files[0]


with open(thread_file, 'rt') as fh:
    file_data = fh.read()
    soup = BeautifulSoup(file_data, "lxml")

sections = soup.find_all('section', class_="not_boxed")
if len(sections) > 1:
    raise ValueError("unexpected multiple <section> elements")
section = sections[0]

for post_div in section.find_all('div', class_='fp'):
    #print(post_div)
    post_info = parse_post(post_div)
    print(post_info)
    print('\n\n\n\n')

{'message_num': 8575306, 'post_num': 1, 'username': 'JayT83', 'edited': 0, 'timestamp': '2024-07-10T10:31:37', 'text': "I owned this book while living in Kansas during the mid to late 1990s and believe it to be written in the 1970s-1980s based on the cover design of the novel. The cover was white with the black boy sitting in profile behind the white boy, who is on his knees facing the reader. The boys are both bound and gagged, with the white boy's arms being bound behind his back.\n\nThe story commences when the two boys are kidnapped when they encounter bank robbers just concluding their heist. One boy starts assigning nicknames to the various men involved based on various characteristics in the absence of actual names, such as the Professor based on his more formal and cultured linguistic patterns.\n\nThe boys are driven several hours out of town and have little knowledge of their location. The kidnappers are amused when the boys' parents are on the news requesting the return of th